# IaaS Windows Logs Review Template - Sentinel
BTV Project Obsidian, 2023.

Author: juju43, https://blueteamvillage.org/programs/project-obsidian/ https://discord.gg/blueteamvillage
<img align="right" width="100" height="100" src="https://cfc.blueteamvillage.org/media/call-for-content-2021/img/20200622_BTVillage_logos_RGB_pos_hcOC7Qx.png">

This playbook is to help validating available logs
It helps to baseline environment, identify gaps and control points.

It targets Defcon31 BlueTeamVillage Project obsidian environment and splunk platform but it can be adapt to other logging platforms.

Possible sources
* Windows Event logs
* sysmon

External sources like EDR or network are intentionally not covered here.

## Resources

* https://github.com/microsoft/msticpy/
* https://infosecjupyterthon.com/
* https://dropbox.tech/security/how-dropbox-security-builds-better-tools-for-threat-detection-and-incident-response
* https://www.malwarearchaeology.com/cheat-sheets
* https://what2log.com/
* https://github.com/SigmaHQ/sigma
* cli usage
  * as is `jupyter run notebook.ipynb --allow-errors` - https://docs.jupyter.org/en/latest/running.html#using-a-command-line-interface
  * with parameters `papermill input.ipynb output.ipynb -p alpha 0.6 -p l1_ratio 0.1` - https://papermill.readthedocs.io/en/latest/usage-workflow.html 

(Win)
* https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/Entity%20Explorer%20-%20Windows%20Host.ipynb
* https://github.com/OTRF/ThreatHunter-Playbook/tree/master/docs/hunts/windows
* https://securitydatasets.com/notebooks/atomic/windows/intro.html
* https://docs.microsoft.com/en-us/windows/security/threat-protection/auditing/audit-user-account-management
* https://learn.microsoft.com/en-us/windows-server/identity/ad-ds/plan/appendix-l--events-to-monitor
* https://learn.microsoft.com/en-us/windows/security/threat-protection/use-windows-event-forwarding-to-assist-in-intrusion-detection
* jpcert: https://jpcertcc.github.io/ToolAnalysisResultSheet/, https://github.com/JPCERTCC/LogonTracer
* https://github.com/mdecrevoisier/Splunk-input-windows-baseline
* https://www.13cubed.com/downloads/windows_event_log_cheat_sheet.pdf

## Findings

_Put your findings here_

## Table of Contents

* Import
* Configuration
* Queries
  * Authentication
    * fail
    * success
  * Remote access
  * Privilege escalation
  * Privileged users activities? root, Administrator...
  * Service activities
    * Time
    * Logging
    * Scheduled tasks
  * System boot, on/off
  * Process activities
  * Network activities
  * File Integrity Monitoring (FIM)
  * AV logs?
  * Web logs?
  * Misc

## Import

In [ ]:
# Check we are running Python 3.6
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

In [ ]:
# Imports
import pandas as pd
import msticpy.nbtools as nbtools
from datetime import datetime,timedelta
import os

In [ ]:
# path to config file
os.environ['MSTICPYCONFIG'] = '/home/ubuntu/msticpyconfig.yaml'
from msticpy.nbtools import *
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
from msticpy.nbtools.data_viewer import DataViewer
from msticpy.vis.matrix_plot import plot_matrix
from msticpy.nbtools import process_tree as ptree
print('Imports Complete')

## Configuration

In [ ]:
# Interactive settings edit
# https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html#using-mpconfigfile-to-check-and-manage-your-msticpyconfig-yaml
from msticpy.config import MpConfigFile, MpConfigEdit, MpConfigControls
mpconfig = MpConfigFile()
# mpconfig.load_default()
# mpconfig.view_settings()
mpconfig

In [ ]:
# q_times = nbwidgets.QueryTime(units='hours', max_before=72, before=1, max_after=0)
#q_times = nbwidgets.QueryTime(origin_time=datetime(2023, 6, 15), units='days', max_before=1, before=0, max_after=0)
q_times = nbwidgets.QueryTime(origin_time=datetime(2023, 6, 15), units='hours', max_before=4, before=0, max_after=0)
q_times.display()

In [ ]:
# If your environment footpring is very large or timeperiod too big, queries not optimized enough may return 'ADX query timed out' or 'Unknown query error' when done through msticpy.
# Ensure to use appropriate filters
query_common_args = ''
# query_common_args = f'''| where _SubscriptionId in ("12345", "67890")'''
results_limit = 10

In [ ]:
query_common_args = query_common_args.strip()
query_common_args = query_common_args + f'''| where TimeGenerated >= datetime({q_times.start})
| where TimeGenerated <= datetime({q_times.end})'''

In [ ]:
# Configuration
qry_prov = QueryProvider("AzureSentinel")

In [ ]:
# Get the default Microsoft Sentinel workspace details from msticpyconfig.yaml
ws_config = WorkspaceConfig()

# Connect to Microsoft Sentinel with our QueryProvider and config details
qry_prov.connect(ws_config)

In [ ]:
# pandas
pd.set_option('display.max_colwidth', 500)

## Queries

### Timeperiod

Let's confirm that we have logs for the targeted timeperiod.

In [ ]:
q_times.start

In [ ]:
q_times.end

In [ ]:
query = f'''SecurityEvent {query_common_args}
| union Event
| summarize max(TimeGenerated),min(TimeGenerated)
'''
df_timeperiod = qry_prov.exec_query(query)
df_timeperiod.head(results_limit)

### Authentication

In [ ]:
query = f'''SecurityEvent {query_common_args}
| where EventID in (4624, 4625, 4634, 4635)
| summarize count() by Computer,EventID
| limit {results_limit}'''
df_auth = qry_prov.exec_query(query)
df_auth.head(results_limit)

### Remote access

In [ ]:
query = f'''SecurityEvent {query_common_args}
| where EventID in (4624, 4625, 4634, 4635) and LogonType in (3)
| summarize count() by Computer,EventID,TargetDomainName,LogonType,LogonTypeName
| limit {results_limit}'''
df_auth = qry_prov.exec_query(query)
df_auth.head(results_limit)

In [ ]:
query = f'''SecurityEvent {query_common_args}
| where EventID in (4624, 4625, 4634, 4635) and LogonType in (8, 10)
| summarize count() by Computer,EventID,TargetDomainName,TargetUserName,LogonType,LogonTypeName
| limit {results_limit}'''
df_auth = qry_prov.exec_query(query)
df_auth.head(results_limit)

### Privilege Escalation

In [ ]:
# https://social.technet.microsoft.com/Forums/en-US/bf693b49-1dd5-45ee-84cf-4a417e5b35ec/run-as-admin-event-log
query = f'''SecurityEvent {query_common_args}
| where EventID in (4688) and TokenElevationType has_any ("1936", "1937", "1938")
| summarize count() by Computer,EventID,TargetDomainName,TokenElevationType,SubjectUserName,TargetUserName,CommandLine,ParentProcessName
| limit {results_limit}'''
df_privesc_cli = qry_prov.exec_query(query)
df_privesc_cli.head(results_limit)

In [ ]:
# Privilege functions: (eventid=4672 or eventid=4673 or eventid=4674)
query = f'''SecurityEvent {query_common_args}
| where EventID in (4672, 4673, 4674)
| summarize count() by Computer,EventID,Activity,SubjectDomainName,SubjectUserName
| limit {results_limit}'''
df_priv_functions = qry_prov.exec_query(query)
df_priv_functions.head(results_limit)

### Services activities

In [ ]:
# general service
query = f'''Event {query_common_args}
| where Source == "Service Control Manager"
| summarize count() by EventID,EventLevel,EventLevelName,RenderedDescription
| sort by EventLevel asc,count_ desc
| limit {results_limit}'''
df_services = qry_prov.exec_query(query)
df_services.head(results_limit)

In [ ]:
# service start/stop
query = f'''Event {query_common_args}
| where Source == "Service Control Manager" and EventID in (7035, 7036)
| summarize count() by EventID,EventLevel,EventLevelName,RenderedDescription
| sort by EventLevel asc,count_ desc
| limit {results_limit}'''
df_services = qry_prov.exec_query(query)
df_services.head(results_limit)

In [ ]:
# time service
# https://learn.microsoft.com/en-us/windows/security/threat-protection/auditing/event-4616
# https://learn.microsoft.com/en-us/windows-server/networking/windows-time-service/windows-time-for-traceability?tabs=257
# https://stackoverflow.com/questions/7852209/is-there-a-way-to-detect-if-the-system-clock-has-been-changed-backwards-in-windo
query = f'''Event {query_common_args}
| where EventID in (4616, 1, 
    257, 258, 259, 260, 261, 262, 263, 264, 265, 266
) and Source != "Microsoft-Windows-Sysmon"
// Source in ("Microsoft-Windows-Kernel-General", "System", "Microsoft-Windows-Time-Service")
| summarize count() by Source,EventLog,EventID,EventLevel,EventLevelName,RenderedDescription
| sort by EventLevel asc,count_ desc
| limit {results_limit}'''
df_services_time = qry_prov.exec_query(query)
df_services_time.head(results_limit)

In [ ]:
# Scheduled Tasks
# https://learn.microsoft.com/en-us/windows/security/threat-protection/auditing/event-4698
query = f'''SecurityEvent {query_common_args}
| where EventID in (4698,4699, 4700, 4701, 4702)
| summarize count() by EventSourceName,EventID,Activity
| sort by count_ desc
| limit {results_limit}'''
df_service_scheduledtasks = qry_prov.exec_query(query)
df_service_scheduledtasks.head(10)

In [ ]:
query = f'''Event {query_common_args}
| where Source == "Microsoft-Windows-TaskScheduler"
| summarize count() by EventLog,EventID,EventLevel,EventLevelName
| limit {results_limit}'''
df_service_scheduledtasks2 = qry_prov.exec_query(query)
df_service_scheduledtasks2.head(10)

In [ ]:
query = f'''Event {query_common_args}
| where EventID == 1102
| summarize count() by EventLog,Source,EventID,EventLevelName,RenderedDescription
| limit {results_limit}'''
df_clearlogs1 = qry_prov.exec_query(query)
df_clearlogs1.head(10)

In [ ]:
query = f'''SecurityEvent {query_common_args}
| where EventID == 1102
| summarize count() by Channel,EventSourceName,EventID,Activity
| limit {results_limit}'''
df_clearlogs2 = qry_prov.exec_query(query)
df_clearlogs2.head(10)

### System boot, on/off

In [ ]:
# https://www.windowscentral.com/how-find-reason-pc-shutdown-no-reason-windows-10
# https://www.shellhacks.com/windows-shutdown-reboot-event-ids-get-logs/
# https://serverfault.com/questions/885601/windows-event-codes-for-startup-shutdown-lock-unlock
query = f'''Event {query_common_args}
| where EventID in (12, 13, 41, 42, 107, 1074, 1076, 6005, 6006, 6008, 6009, 6013)
| summarize count() by EventLog,EventID,EventLevel,EventLevelName
| limit {results_limit}'''
df_system_onoff = qry_prov.exec_query(query)
df_system_onoff.head(10)

### Error, warnings

In [ ]:
query = f'''Event {query_common_args}
| where EventLevelName in ("Error")
| summarize count() by Source,EventID,EventLevel,EventLevelName
| sort by count_ desc
| limit {results_limit}
'''
df_errors = qry_prov.exec_query(query)
df_errors.head(results_limit)

In [ ]:
query = f'''Event {query_common_args}
| where EventLevelName in ("Warning")
| summarize count() by Source,EventID,EventLevel,EventLevelName
| sort by count_ desc
| limit {results_limit}
'''
df_warn = qry_prov.exec_query(query)
df_warn.head(results_limit)

### Process activities

In [ ]:
query = f'''SecurityEvent {query_common_args}
| where EventID==4688
| summarize count() by EventSourceName,EventID,SubjectDomainName,SubjectUserName,CommandLine
| sort by count_ desc
| limit {results_limit}
'''
df_process4688 = qry_prov.exec_query(query)
df_process4688.head(results_limit)

In [ ]:
query = f'''Event {query_common_args}
| where Source == "Microsoft-Windows-Sysmon" and EventID==1
| summarize count() by EventLog,EventID'''
df_process1 = qry_prov.exec_query(query)
df_process1.head(10)

### Network activities

In [ ]:
query = f'''Event {query_common_args}
| where Source == "Microsoft-Windows-Sysmon" and EventID==3
| summarize count() by EventLog,EventID'''
df_network1 = qry_prov.exec_query(query)
df_network1.head(10)

### File Integrity Monitoring

In [ ]:
# https://learn.microsoft.com/en-us/windows/security/threat-protection/auditing/basic-audit-object-access
# https://learn.microsoft.com/en-us/windows/security/threat-protection/auditing/event-4660
# https://learn.microsoft.com/en-us/windows/security/threat-protection/auditing/event-4663
query = f'''SecurityEvent {query_common_args}
| where EventID in (4656, 4658, 4659, 4660, 4661, 4663, 4715, 4907)
| summarize count() by Channel,EventID,Activity,ProcessName
| limit {results_limit}'''
df_objectaudit = qry_prov.exec_query(query)
df_objectaudit.head(results_limit)

### Web logs

Prefer separated notebook.

In [ ]:
query = f'''W3CIISLog {query_common_args}
| summarize count() by Computer,sSiteName,sIP,cIP,csMethod,scStatus,csUriStem
| sort by count_ desc
| limit {results_limit}'''
df_iis = qry_prov.exec_query(query)
df_iis.head(results_limit)

### Misc

In [ ]:
# heartbeat, agent and version (any outdated?)
query = f'''Heartbeat {query_common_args}
| summarize count() by ResourceGroup,Computer,Category,SCAgentChannel,OSType,Version
| sort by Computer
| render table
| limit {results_limit}'''
df_heartbeat = qry_prov.exec_query(query)
df_heartbeat.head(results_limit)

In [ ]:
# Noise?
# https://answers.microsoft.com/en-us/windows/forum/windows_10-performance/excessive-security-log-events-event-id-5379/8eb0c350-ce2f-4521-9cfd-f7b816d54715
query = f'''SecurityEvent {query_common_args}
| summarize count() by EventSourceName,EventID
| sort by count_ desc
| limit {results_limit}'''
df_volume = qry_prov.exec_query(query)
df_volume.head(results_limit)

In [ ]:
query = f'''Event {query_common_args}
| summarize count() by Source,EventID
| sort by count_ desc
| limit {results_limit}'''
df_volume2 = qry_prov.exec_query(query)
df_volume2.head(results_limit)

In [ ]:
# Partly covered by above
SecurityEventId = [
    # auth
    "4624",
    "4625",
    # sysmon
    "1",
    "3",
    "17",
    "18",
    "20",
    "21",
    "22",
    "25",
    # process commandline
    "4688",
    # scheduled task,
    "4698",
    # https://docs.microsoft.com/en-us/windows/security/threat-protection/auditing/event-4697
    "4697",
    "5140",
    "5145",
    "4672",
    "4104",
    "1000",
    # less usual ones
    # https://docs.microsoft.com/en-us/windows/security/threat-protection/auditing/audit-user-account-management
    "4720",
    "4722",
    "4723",
    "4724",
    "4725",
    "4726",
    "4738",
    "4740",
    "4765",
    "4766",
    "4767",
    "4780",
    "4781",
    "4794",
    "4798",
    "5376",
    "5377",
    # https://docs.microsoft.com/en-us/windows-server/identity/ad-ds/plan/appendix-l--events-to-monitor
    "4715",
    "4907",
    "4656",
    "4658",
    "4659",
    "4661",
    "4660",
    # Policy change: eventid=4739
    "4739",
    # Privilege functions:
    "4672",
    "4673",
    "4674",
]

In [ ]:
query = f'''SecurityEvent {query_common_args}
| where EventID in ({','.join(SecurityEventId)})
| summarize count() by Channel,EventID,Activity,ProcessName
| limit {results_limit}'''
df_list_securityeventid = qry_prov.exec_query(query)
df_list_securityeventid.head(results_limit)

In [ ]:
query = f'''Event {query_common_args}
| where EventID in ({','.join(SecurityEventId)})
| summarize count() by Source,EventID
| limit {results_limit}'''
df_list_eventid = qry_prov.exec_query(query)
df_list_eventid.head(results_limit)

In [ ]:
# ASIM tables?
# https://learn.microsoft.com/en-us/azure/sentinel/normalization-schema-audit
# https://learn.microsoft.com/en-us/azure/sentinel/normalization-schema-authentication
# and so on

In [ ]:
# PII, credentials detection? many more variants...
# sysmon and powershell logging likely to catch many cli creds.
query = f'''search in (Event, SecurityEvent) ("--password" or "password=" or "_PASSWORD" or "PASSWORD_" or "credentials=" or "pin=" or "cvv=" or "hl7-org") {query_common_args}
| summarize count() by _SubscriptionId,$table,EventID
| sort by count_ desc 
| limit {results_limit}'''
df_sensitivedata = qry_prov.exec_query(query)
df_sensitivedata.head(results_limit)